In [90]:
import pandas as pd
import json

In [91]:
df = pd.read_csv("server/data/actes-criminels.csv")

In [92]:
len(df)

244298

In [93]:
len(df) - len(df.dropna())

41425

In [94]:
df = df.dropna()

In [95]:
df.head()

,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE
0,Vol de véhicule à moteur,2018-09-13,jour,30.0,294904.159001,5.047549e+06,-73.626778,45.567780
1,Vol de véhicule à moteur,2018-04-30,jour,30.0,294904.159001,5.047549e+06,-73.626778,45.567780
2,Vol de véhicule à moteur,2018-09-01,nuit,7.0,290274.565000,5.042150e+06,-73.685928,45.519122
6,Méfait,2017-07-30,jour,38.0,297654.715002,5.041877e+06,-73.591457,45.516776
8,Vol dans / sur véhicule à moteur,2017-08-01,jour,39.0,294259.780993,5.051450e+06,-73.635117,45.602873


In [96]:
df["DATE"].min()

'2015-01-01'

In [97]:
df["DATE"].max()

'2023-02-01'

In [98]:
df["DATE"] = pd.to_datetime(df["DATE"])

In [99]:
df["QUART"].unique()

array(['jour', 'nuit', 'soir'], dtype=object)

In [100]:
df["CATEGORIE"].unique()

array(['Vol de véhicule à moteur', 'Méfait',
       'Vol dans / sur véhicule à moteur', 'Introduction',
       'Vols qualifiés', 'Infractions entrainant la mort'], dtype=object)

In [101]:
only_2023 = df[df["DATE"].dt.year >= 2023]

In [112]:
january_2023 = only_2023[only_2023["DATE"] <= "2023-01-31"]
last_2_weeks = january_2023[january_2023["DATE"] >= "2023-01-16"]

In [111]:
last_2_weeks["CATEGORIE"].unique()

array(['Vol de véhicule à moteur', 'Vol dans / sur véhicule à moteur',
       'Méfait', 'Introduction', 'Vols qualifiés',
       'Infractions entrainant la mort'], dtype=object)

In [113]:
last_2_weeks.to_csv("january_last_2.csv")

In [19]:
only_2022 = df[df["DATE"].dt.year >= 2022]

In [20]:
len(only_2022)

29098

In [22]:
len(only_2022[only_2022["DATE"].dt.month == 12])

2118

In [23]:
december_2022 = only_2022[only_2022["DATE"].dt.month == 12]

In [25]:
december_2022["CATEGORIE"].unique()

array(['Vol de véhicule à moteur', 'Introduction',
       'Vol dans / sur véhicule à moteur', 'Méfait', 'Vols qualifiés',
       'Infractions entrainant la mort'], dtype=object)

In [26]:
december_2022.to_csv("server/data/december_2022_crimes")

In [61]:
def get_crimes_by_field(df, column, values):
    eligible = df[df[column].isin(values)]
    return eligible.apply(lambda x: x.to_json(), axis=1)

In [79]:
def get_crimes_by_date(df, start, end):
    
    eligible = df.loc[(df['DATE'] >= start)
                     & (df['DATE'] <= end)]
    return eligible

In [62]:
def get_crimes(quarts = None,  categories = None, start= "2023-02-01", end = "2015-01-01"):
    result = get_crimes_by_date(pd.read_csv(default_dataset), start, end)
    if quarts is not None:
        result = get_crimes_by_field(result, "QUART", quarts)
    if categories is not None:
        result = get_crimes_by_date(result, "CATEGORIE", categories)
    return result.apply(lambda x: x.to_json(), axis=1)

In [81]:
dated = get_crimes_by_date(december_2022, "2022-12-10", "2022-12-15")
get_crimes_by_field(dated, "QUART", ["J"])

213589    {"CATEGORIE":"Introduction","DATE":16706304000...
213590    {"CATEGORIE":"M\u00e9fait","DATE":167063040000...
213598    {"CATEGORIE":"Introduction","DATE":16707168000...
213752    {"CATEGORIE":"M\u00e9fait","DATE":167071680000...
213753    {"CATEGORIE":"M\u00e9fait","DATE":167071680000...
                                ...                        
244116    {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
244122    {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
244124    {"CATEGORIE":"M\u00e9fait","DATE":167080320000...
244125    {"CATEGORIE":"M\u00e9fait","DATE":167080320000...
244127    {"CATEGORIE":"Vol dans \/ sur v\u00e9hicule \u...
Length: 187, dtype: object

In [51]:
jsons = get_crimes_by_field("CATEGORIE", ["Méfait", "Vol de véhicule à moteur"])

In [52]:
jsons

0         {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
1         {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
2         {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
6         {"CATEGORIE":"M\u00e9fait","DATE":150137280000...
10        {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
                                ...                        
244280    {"CATEGORIE":"M\u00e9fait","DATE":166674240000...
244281    {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
244287    {"CATEGORIE":"Vol de v\u00e9hicule \u00e0 mote...
244292    {"CATEGORIE":"M\u00e9fait","DATE":166717440000...
244293    {"CATEGORIE":"M\u00e9fait","DATE":166691520000...
Length: 79986, dtype: object

In [56]:
get_crimes_by_date("2022-01-01","2022-02-01")

,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE
188276,Introduction,2022-01-25,jour,12.0,297357.344006,5.038477e+06,-73.595212,45.486178
193746,Vol dans / sur véhicule à moteur,2022-01-13,jour,46.0,300267.884000,5.053248e+06,-73.558115,45.619119
193747,Introduction,2022-01-19,jour,20.0,299287.578006,5.040727e+06,-73.570544,45.506444
193755,Méfait,2022-01-31,jour,12.0,298561.630996,5.039475e+06,-73.579818,45.495166
195073,Vol de véhicule à moteur,2022-01-02,jour,44.0,298653.985002,5.045337e+06,-73.578710,45.547920
...,...,...,...,...,...,...,...,...
235141,Vol de véhicule à moteur,2022-01-31,soir,7.0,287594.124990,5.038881e+06,-73.720124,45.489641
235142,Introduction,2022-02-01,jour,49.0,305338.435000,5.055484e+06,-73.493093,45.639256
235143,Vol dans / sur véhicule à moteur,2022-02-01,jour,1.0,271603.688000,5.033208e+06,-73.924310,45.438021
235144,Introduction,2022-02-01,jour,9.0,296340.618010,5.036678e+06,-73.608188,45.469981
